In [9]:
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
from keras.models import load_model
import json
import random
import tkinter
from tkinter import *


In [10]:
def clean_up_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [11]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
   
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result




In [12]:
def bow(sentence, words, show_details=True):
    sentence_words=clean_up_sentence(sentence)

    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i]=1
                if show_details:
                    print("Found in bag: %s" %w)
            
    return np.array(bag)
    

In [13]:
def predict_class(sentence):
    words = pickle.load(open(r'C:\Users\mushirih\Desktop\projs\jupyter\May20\words.pkl','rb'))
    classes = pickle.load(open(r'C:\Users\mushirih\Desktop\projs\jupyter\May20\classes.pkl','rb'))
    model = load_model(r'C:\Users\mushirih\Desktop\projs\jupyter\May20\chatbot_model.h5')

    p = bow(sentence,words, show_details=True)
    res = model.predict(np.array([p]))[0]
    print('1',model.predict(np.array([p])))
    ERROR_THRESHOLD=.25
    results = [[i,r] for i, r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent":classes[r[0]],"probability":str(r[1])})
        print('-->',"intent",classes[r[0]],"probability",str(r[1]),'<--')
    return return_list

In [14]:
def chatbot_response(text):
    
    
    intents = json.loads(open(r'C:\Users\mushirih\Desktop\projs\jupyter\data\python-project-chatbot-codes\intents.json').read())

    ints = predict_class(text)
    print(ints)
    res=getResponse(ints,intents)
    
    return res

In [15]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
    if msg !='':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END,"You: "+msg+'\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana",12))
        res = chatbot_response(msg)
        ChatLog.insert(END,"Bot: "+res+'\n\n')
        
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

In [16]:



    
    
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

ChatLog = Text(base,bd=0,bg='white',height='8', width='50', font='Arial')

ChatLog.config(state=DISABLED)

scrollbar = Scrollbar(base, command=ChatLog.yview, cursor='heart')
ChatLog['yscrollcommand']=scrollbar.set

SendButton = Button(base, font=("Verdana",12,'bold'),text='Send',width="12",height="5",bd="0",bg="#32de97",activebackground="#3c9d9b",fg="#ffffff",command=send)

EntryBox = Text(base,bd=0,bg='white',width=29,height=5,font="Arial")

scrollbar.place(x=376,y=6,height=386)
ChatLog.place(x=6,y=6,height=386)
EntryBox.place(x=128,y=401,height=90, width=265)
SendButton.place(x=6,y=401,height=90)

base.mainloop()

Found in bag: hello
1 [[7.2517429e-07 7.6474016e-06 3.1774594e-05 1.7351897e-05 9.9973911e-01
  2.0270274e-05 1.1845859e-05 4.8965845e-05 1.2238864e-04]]
--> intent greeting probability 0.9997391 <--
[{'intent': 'greeting', 'probability': '0.9997391'}]
Found in bag: you
Found in bag: good
1 [[2.8652395e-04 1.2567117e-03 1.3648621e-03 2.1949137e-02 6.6715491e-01
  1.0321814e-03 2.8325947e-02 3.1924574e-04 2.7831054e-01]]
--> intent greeting probability 0.6671549 <--
--> intent thanks probability 0.27831054 <--
[{'intent': 'greeting', 'probability': '0.6671549'}, {'intent': 'thanks', 'probability': '0.27831054'}]
Found in bag: pharmacy
1 [[9.7039856e-06 6.2506376e-07 2.8621612e-06 1.2705604e-05 6.3439627e-05
  5.3565589e-07 8.6454520e-06 9.9990141e-01 5.1813180e-09]]
--> intent pharmacy_search probability 0.9999014 <--
[{'intent': 'pharmacy_search', 'probability': '0.9999014'}]
1 [[0.00089469 0.00176459 0.00235565 0.0626983  0.8743568  0.00604719
  0.00350324 0.00366184 0.04471774]]
--> 